In [37]:
import os
import pandas as pd
import datetime
import glob
import numpy as np

# Lectura del archivo CSV 'metadata_sample_submission.csv'
sub = pd.read_csv('metadata_sample_submission.csv')

# Añadir una columna 'year' con valor 2023
sub['year'] = 2023

# Combinar las columnas 'month', 'day', 'hour', 'year' en una columna de tipo datetime 'time'
sub['time'] = pd.to_datetime(sub[['month', 'day', 'hour', 'year']])

# Obtener los valores de año, mes, hora, día del año, hora del año, día de la semana y fecha sin hora
sub['year'] = sub.time.dt.year
sub['month'] = sub.time.dt.month
sub['hour'] = sub.time.dt.hour
sub['dayyear'] = sub.time.dt.day_of_year
sub['houryear'] = (sub.dayyear - 1) * 24 + sub.hour
sub['dayofweek'] = sub['time'].dt.dayofweek
sub['time2'] = pd.to_datetime(sub['time'].dt.date)

# Lectura del archivo CSV 'Precipitacio.csv', igual que antes añadimos la informacion de la lluvia y el viento
df = pd.read_csv('Precipitacio.csv')

# Eliminar columnas no necesarias y crear columnas con variables de tiempo
df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
df['year'] = df.time.dt.year
df['dayyear'] = df.time.dt.day_of_year

# Agrupar por año y día del año y sumar los valores de VALOR_LECTURA
df = df.groupby(['year', 'dayyear']).sum()

# Crear una columna 'Rain' que indica si ha llovido ese día (1 si VALOR_LECTURA > 1, 0 en caso contrario)
df['Rain'] = np.where(df['VALOR_LECTURA'] > 1, 1, 0)

# Crear una columna 'Rain_Lectura' con el valor real de la lectura de lluvia en esa hora
df['Rain_Lectura'] = df['VALOR_LECTURA']

# Eliminar la columna 'VALOR_LECTURA'
df = df.drop(['VALOR_LECTURA'], axis=1)

# Reiniciar el índice del DataFrame
df = df.reset_index()

# Crear una columna 'time2' que combina el año y el día del año en un formato de fecha
df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')

# Combinar la información del DataFrame 'sub' con el DataFrame 'df' utilizando la columna 'time2'
sub = pd.merge_ordered(sub, df, how='left', on=['time2'], fill_method='ffill')

# Eliminar las columnas redundantes de 'year_y' y 'dayyear_y'
sub = sub.drop(['year_y', 'dayyear_y'], axis=1)

# Lectura del archivo CSV 'Vent.csv'
df = pd.read_csv('Vent.csv')

# Eliminar columnas no necesarias y crear columnas con variables de tiempo
df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
df['year'] = df.time.dt.year
df['dayyear'] = df.time.dt.day_of_year

# Agrupar por año y día del año y obtener el valor máximo de VALOR_LECTURA
df = df.groupby(['year', 'dayyear']).max()

# Crear una columna 'Wind' que indica si hubo viento ese día (1 si VALOR_LECTURA > 20, 0 en caso contrario)
df['Wind'] = np.where(df['VALOR_LECTURA'] > 20, 1, 0)

# Crear una columna 'Wind_Lectura' con el valor real de la lectura de viento en esa hora
df['Wind_Lectura'] = df['VALOR_LECTURA']

# Eliminar las columnas 'VALOR_LECTURA', 'DATA_LECTURA' y 'time'
df = df.drop(['VALOR_LECTURA', 'DATA_LECTURA', 'time'], axis=1)

# Reiniciar el índice del DataFrame
df = df.reset_index()

# Crear una columna 'time2' que combina el año y el día del año en un formato de fecha
df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')

# Combinar la información del DataFrame 'sub' con el DataFrame 'df' utilizando la columna 'time2'
sub = pd.merge_ordered(sub, df, how='left', on=['time2'], fill_method='ffill')

# Ordenar el DataFrame por el índice
sub = sub.sort_values(by='index')

# Guardar el DataFrame resultante en un archivo CSV llamado 'submission.csv'
sub.to_csv('submission.csv', index=False)


C:\Users\xiscu\AppData\Local\Temp\ipykernel_10796\496169133.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df=df.groupby(['year','dayyear']).sum()
